In [16]:
import pandas as pd
import os
import numpy as np
from enrich2.variant import WILD_TYPE_VARIANT
import glob
#reference:
#https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#hdf5-pytables

In [17]:
#If you get a dependency error that tables is not installed, try sudo apt-get install python3-tables, that solved it for me

In [18]:
results_path = "/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/"

In [19]:
#list all hdf5 files
listing = glob.glob(os.path.join(results_path, 'hdf5_manipulation_test', '*h5'))
listing

['/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2B0_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2A3_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2B2_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2A0_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2C2_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2B1_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2B3_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_test/2C1_lib.h5',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/hdf5_manipulation_t

In [20]:
folders = glob.glob(os.path.join(results_path, 'tile*_all_reps_minvarcount10_corr_sy'))
folders

['/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile5_all_reps_minvarcount10_corr_sy',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile4_all_reps_minvarcount10_corr_sy',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile1_all_reps_minvarcount10_corr_sy',
 '/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile2_all_reps_minvarcount10_corr_sy']

In [21]:
for folder in folders:
    listing = glob.glob(os.path.join(folder, '*h5'))
    for h_file in listing:
        print(h_file)
        my_store = pd.HDFStore(h_file)

        #get the synonymous to WT counts
        syn_counts = int(my_store.select('/main/synonymous/counts', where="index in ['_sy']")['count'])
        print('sy:',syn_counts)

        #rm wt counts from /main/variants/counts and /main/synonymous/counts
        my_store.remove('/main/synonymous/counts', where="index in ['_wt']")
        my_store.remove('/main/variants/counts', where="index in ['_wt']")
        my_store.remove('/raw/variants/counts', where="index in ['_wt']")

        #re-add a wt var with the syn_counts
        my_df = pd.DataFrame(data=syn_counts, index=['_wt'], columns=['count'], dtype=np.int32, copy=False)
        my_store.append('/main/synonymous/counts', my_df)
        my_store.append('/main/variants/counts', my_df)
        my_store.append('/raw/variants/counts', my_df)

        #close the storage to save
        my_store.close()
    

/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3C0_lib.h5
('sy:', 8327)
/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3C2_lib.h5
('sy:', 10170)
/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3B0_lib.h5
('sy:', 8946)
/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3A0_lib.h5
('sy:', 9995)
/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3B3_lib.h5
('sy:', 17304)
/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3C3_lib.h5
('sy:', 7061)
/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile3_all_reps_minvarcount10_corr_sy/3B2_lib.h5
('sy:', 12684)
/home/henrike/Documents/PD_AS/projects

In [ ]:
#verify

In [22]:
my_store = pd.HDFStore('/home/henrike/Documents/PD_AS/projects/Sofie_Mave/results/enrich_full_dhfr/tile1_all_reps_minvarcount10_corr_sy/1A1_lib.h5')

In [23]:
my_store["/main/variants/counts"].head()

,count
c.68T>G (p.Leu23Trp),8939
"c.68T>A (p.Leu23Tyr), c.69G>T (p.Leu23Tyr)",8639
"c.67T>G (p.Leu23Glu), c.68T>A (p.Leu23Glu), c.69G>A (p.Leu23Glu)",4561
"c.94T>G (p.Phe32Asp), c.95T>A (p.Phe32Asp)",2830
"c.22A>T (p.Ile8Phe), c.24C>T (p.Ile8Phe)",2611


In [24]:
my_store.select('/main/synonymous/counts', where="index in ['_wt']")

,count
_wt,2717


In [25]:
my_store.select('/main/synonymous/counts', where="index in ['_sy']")

,count
_sy,2717


In [26]:
my_store.select('/main/variants/counts', where="index in ['_wt']")

,count
_wt,2717


In [27]:
my_store.keys()

['/raw/filter',
 '/raw/overlap_mismatches',
 '/raw/variants/counts',
 '/main/variants/counts',
 '/main/synonymous/counts']

In [28]:
my_store.select('/raw/variants/counts', where="index in ['_wt']")

,count
_wt,2717


In [15]:
my_store['/raw/overlap_mismatches'].head()

,resolved,unresolved,first
28,11392,4562,15954
29,9526,3532,2637
30,8458,3931,4831
31,10815,6861,4324
32,11410,4774,2179


In [ ]:
#old: trying it out on one storage

In [ ]:
#open the hdf5 storage
#if we open the experiment level one tile2_all_reps_exp.h5 the counts in /main/variants/counts are all NA
#this is a known bug that was reported before.
#I might have to change the wt counts for each selection anyway. Or for each lib?
#Alan said:
#1. Delete the selection and experiment HDF5 files.
#2. For each SeqLib HDF5 file, replace the '_wt' count with the '_sy' count in the /main/counts table
my_store = pd.HDFStore(os.path.join(results_path, "hdf5_manipulation_test/2B0_lib.h5"))

In [ ]:
my_store.keys()

In [ ]:
#this shows the /main/variants/counts table
#an equivalent method is my_store.get('/main/variants/counts')
my_store["/main/variants/counts"].head()

In [ ]:
my_store["/main/synonymous/counts"].head()

In [ ]:
#does not work :(
#my_store["/main/variants/counts"]['_wt']
#neither does this
#my_store["/main/synonymous/counts/_wt"]
#this type of selection using where works though!
c = my_store.select('/main/synonymous/counts', where="index in ['_sy']")

In [ ]:
#I'll try to remove and re-add the _wt count
my_store.remove('/main/synonymous/counts', where="index in ['_wt']")

In [ ]:
#display result
my_store["/main/synonymous/counts"].head()

In [ ]:
#now re-add the wt var with the sy count
my_df = pd.DataFrame(data=int(c['count']), index=['_wt'], columns=['count'], dtype=np.int32, copy=False)
my_df.head()

In [ ]:
my_store.append('/main/synonymous/counts', my_df)

In [ ]:
#display result
my_store["/main/synonymous/counts"].head()

In [ ]:
my_store.select('/main/synonymous/counts', where="index in ['_wt']")

In [ ]:
my_store.select('/main/synonymous/counts', where="index in ['_sy']")

In [ ]:
#close the storage to save
my_store.close()